In [2]:
import pandas as pd
import seaborn as sns
import copy
import matplotlib
import plotly.express as px
from PIL import Image
import matplotlib.pyplot as plt
from pylab import rcParams
# Reglas de Asociación
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Análisis

In [3]:
df_items = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_items.csv")
df_orders = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_orders.csv")

In [4]:
df_items.head()

,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75
4,It005,HDR-FLT,Flat White,Hot Drinks,NaN,3.15


In [5]:
df_orders.head()

,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out


In [6]:
print(df_orders.shape[0])

521


In [7]:
df_orders.order_id.duplicated().sum()

91

In [8]:
df_orders[df_orders['quantity']<=0]

,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out


# columna de descripción

In [9]:
df_items['descripcion'] =df_items.apply(lambda x: str(x['item_name']) if x['item_size']!= x['item_size'] else  str(x['item_name']) + ' | ' + str(x['item_size']) , axis=1)

In [10]:
df_items.head(5)

,item_id,sku,item_name,item_cat,item_size,item_price,descripcion
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45,Cappuccino | Medium
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75,Cappuccino | Large
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45,Latte | Medium
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75,Latte | Large
4,It005,HDR-FLT,Flat White,Hot Drinks,NaN,3.15,Flat White


In [11]:
df_orders = pd.merge(df_orders,df_items[['item_id','descripcion']],on='item_id',how='left')

# reglas de asociación

In [12]:
# Quitamos los ítems DOTCOM POSTAGE y POSTAGE porque no son productos.
# Llenamos con 0 los pedidos donde un producto no fué comprado
df_group = (df_orders.groupby(['order_id', 'descripcion'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('order_id'))
# Seteamos True/False dependiendo de cada valor
df_group = df_group.applymap(lambda x: True if x >0 else False)
df_group

C:\Users\facuv\AppData\Local\Temp\ipykernel_11216\2367238087.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_group = df_group.applymap(lambda x: True if x >0 else False)


descripcion,Cappuccino | Large,Cappuccino | Medium,Caramel Macchiato | Large,Caramel Macchiato | Medium,Cold Coffee | Large,Cold Coffee | Medium,Cold Mocha | Large,Cold Mocha | Medium,Espresso,Flat White,...,Latte | Large,Latte | Medium,Lemonade | Large,Lemonade | Medium,Mocha | Large,Mocha | Medium,Sandwich Ham&Cheese,Sandwich Salami&Mozzarella,White Mocha | Large,White Mocha | Medium
order_id,,,,,,,,,,,,,,,,,,,,,
ORD001,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
ORD002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ORD003,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
ORD004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ORD005,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ORD432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
ORD433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
ORD434,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
x = 0.005
frequent_itemsets = apriori(df_group, min_support=x, use_colnames=True)
frequent_itemsets.sort_values(by="support", ascending=False)

,support,itemsets
6,0.069409,(Cold Mocha | Large)
9,0.069409,(Flat White)
19,0.066838,(Mocha | Medium)
10,0.056555,(Hot Chocolate | Large)
23,0.056555,(White Mocha | Medium)
1,0.053985,(Cappuccino | Medium)
12,0.053985,(Iced Tea | Large)
16,0.053985,(Lemonade | Large)
22,0.051414,(White Mocha | Large)
3,0.051414,(Caramel Macchiato | Medium)


In [14]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
# Ordenamos por confianza de mayor a menor
rules.sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
6,(Lemonade | Large),(Cold Mocha | Large),0.053985,0.069409,0.012853,0.238095,3.430335,1.0,0.009106,1.221401,0.748913,0.116279,0.181268,0.211640
22,(Sandwich Ham&Cheese),(White Mocha | Medium),0.041131,0.056555,0.007712,0.187500,3.315341,1.0,0.005386,1.161163,0.728329,0.085714,0.138794,0.161932
7,(Cold Mocha | Large),(Lemonade | Large),0.069409,0.053985,0.012853,0.185185,3.430335,1.0,0.009106,1.161019,0.761326,0.116279,0.138688,0.211640
12,(Sandwich Salami&Mozzarella),(Espresso),0.048843,0.048843,0.007712,0.157895,3.232687,1.0,0.005326,1.129499,0.726126,0.085714,0.114651,0.157895
13,(Espresso),(Sandwich Salami&Mozzarella),0.048843,0.048843,0.007712,0.157895,3.232687,1.0,0.005326,1.129499,0.726126,0.085714,0.114651,0.157895


# Consigna
- conceptualmente que representa la metrica 'support' y que representa la metrica 'confidence'
- obtener las reglas de asociacion usando item_name en lugar de la descripcion
- obtener las reglas de asociacion del siguiente dataset:<br>
http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

pueden usar pd.read_excel() para archivos xls o xlsx


In [ ]:
df2 = pd.read_excel( "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx" )